In [1]:

import os
import pandas as pd


In [2]:

# Pedir al usuario la carpeta raíz
carpeta_raiz = input("Ingresa la ruta de la carpeta raíz: ").strip()

# Buscar el único archivo Excel en la carpeta raíz
excel_files = [f for f in os.listdir(carpeta_raiz) if f.endswith('.xlsx') or f.endswith('.xls')]

if len(excel_files) != 1:
    print("⚠️ No se encontró exactamente un archivo Excel en la carpeta raíz.")
    exit()

# Cargar el archivo Excel encontrado
ruta_excel = os.path.join(carpeta_raiz, excel_files[0])
df = pd.read_excel(ruta_excel)

# Asegurar que las columnas estén bien nombradas
df.columns = [col.strip().upper() for col in df.columns]
df = df.rename(columns={
    "DOCUMENTO": "DOCUMENTO",
    "ASEGURADORA": "ASEGURADORA",
    "PROGRAMA": "PROGRAMA",
    "APELLIDOS Y NOMBRES": "NOMBRE_COMPLETO"
})

# Convertir documento a texto por si hay ceros a la izquierda
df["DOCUMENTO"] = df["DOCUMENTO"].astype(str)


In [3]:

# Obtener subcarpetas dentro de la carpeta raíz (cada una representa un tipo de soporte)
subcarpetas = [carpeta for carpeta in os.listdir(carpeta_raiz)
               if os.path.isdir(os.path.join(carpeta_raiz, carpeta))]

# Crear columnas nuevas por cada subcarpeta
df_rutas = df.copy()
for sub in subcarpetas:
    df[sub] = ""        # Para reporte OK
    df_rutas[sub] = ""  # Para reporte con rutas


In [4]:

# Buscar archivos PDF por subcarpeta
for sub in subcarpetas:
    ruta_sub = os.path.join(carpeta_raiz, sub)
    archivos = os.listdir(ruta_sub)
    archivos_pdf = [f for f in archivos if f.lower().endswith('.pdf')]

    for idx, fila in df.iterrows():
        doc_paciente = fila["DOCUMENTO"]
        encontrado = None
        for archivo in archivos_pdf:
            if doc_paciente in archivo.replace(" ", "").upper():
                ruta_pdf = os.path.join(ruta_sub, archivo)
                df.at[idx, sub] = "OK"
                df_rutas.at[idx, sub] = ruta_pdf
                break


In [5]:

# Guardar ambos DataFrames en un archivo Excel con dos hojas
archivo_salida = os.path.join(carpeta_raiz, "reporte_completo_con_rutas.xlsx")
with pd.ExcelWriter(archivo_salida, engine='openpyxl') as writer:
    df.to_excel(writer, index=False, sheet_name="REPORTE_OK")
    df_rutas.to_excel(writer, index=False, sheet_name="REPORTE_RUTAS")

print("✅ Archivo generado con éxito:", archivo_salida)


✅ Archivo generado con éxito: C:\Users\AUX ADMINISTRATIVO\Documents\TOBAR\BOOTCAMP PROGRAMACION\Phyton proyecto pruebas\07 JULIO\reporte_completo_con_rutas.xlsx
